## Lidiar con datos faltantes

### Valores nulos

Es muy común que nuestros DataFrames presenten datos faltantes, antes de empezar a procesar nuestros DataFrames veamos un poco en qué consisten los objetos NaN (Not a Number).

Importemos las librerias Pandas y Numpy para esto:

In [1]:
import numpy as np
import pandas as pd

Un número que no está definido usualmente se representa como NaN: Not a Number

In [2]:
np.nan

nan

¡Este objeto tiene propiedades matemáticas! Al sumar un número, obtenemos como
respuesta el mismo NaN.


In [3]:
np.nan + 0

nan

In [4]:
np.nan > 0

False


La versión 1.0 de pandas incluye un nuevo objeto NA, que es mucho más general pues, ademas de interactuar con números, tambien puede hacerlo con cadenas de texto u otras varaibles como las de tipo booleano. Si quieres que esta nueva definición este incluida entre tus cálculos usa:
```python
pd.options.mode.use_inf_as_na = True
```

Lo que hace esta función es tratar *None, Nan, inf, NA* como **NA**

Al sumar NA a una cadena de texto, obtengo el mismo NA:

In [42]:
pd.DataFrame([np.inf, 2, 3, np.inf]).isnull()

,0
0,False
1,False
2,False
3,False


convirtiendo en 0 los objetos **NONE**

In [43]:
list_from_source_code = [None, 2, 3, 4, None, 6, 7]
clean_list_from_source_code = [0 if i is None else i for i in list_from_source_code]

In [48]:
pd.DataFrame(list_from_source_code).isna() #igual si usas isnull

,0
0,True
1,False
2,False
3,False
4,True
5,False
6,False


In [44]:
clean_list_from_source_code

[0, 2, 3, 4, 0, 6, 7]

In [16]:
pd.NA +'Hola mundo'

<NA>

In [17]:
pd.NA | False

<NA>

A continuación, vamos a crear un DataFrame

In [52]:
df = pd.DataFrame(np.arange(0, 15).reshape(5, 3), columns=['a', 'b', 'c'])
df

,a,b,c
0,0,1,2
1,3,4,5
2,6,7,8
3,9,10,11
4,12,13,14


In [53]:
#Y vamos a añadir algunas variables no definidas:

df['d'] = np.nan
df['e'] = np.arange(15, 20)
df.loc[5,:] = pd.NA
df.loc[4,'a'] = pd.NA
df.loc[0,'d'] = 1
df.loc[5,'d'] = 10
df

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,NaN,16.0
2,6.0,7.0,8.0,NaN,17.0
3,9.0,10.0,11.0,NaN,18.0
4,<NA>,13.0,14.0,NaN,19.0
5,<NA>,<NA>,<NA>,10,<NA>


Para reconocer cuando un objeto es nulo simplmente usamos:

In [54]:
df.isnull()

,a,b,c,d,e
0,False,False,False,False,False
1,False,False,False,True,False
2,False,False,False,True,False
3,False,False,False,True,False
4,True,False,False,True,False
5,True,True,True,False,True


En dónde todas nuestras variables no definidas fueron marcadas con TRUE,
`df.isna()` tambien cumple esta función.

Conocer el número de variables nulas por columna puede hacerse juntando el
comando anterior con la funcion de suma:


In [10]:
df.isnull().sum()

a    2
b    1
c    1
d    4
e    1
dtype: int64

Si lo que nos interesa es conocer el número de variables núlas por filas, basta
con usar axis=1:


In [22]:
df.isnull().sum(axis=1)

0    0
1    1
2    1
3    1
4    2
5    4
dtype: int64

O todos los elementos nulos de nuestro DataFrame:

In [23]:
df.isnull().sum().sum()


9

Reconocer estos elementos nos puede ayudar a filtrar en nuestro DataFrame, en
este caso, me gustaría filtrar por las variables no nulas de la columna ‘a’:


In [55]:
df[df['a'].notnull()]


,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,NaN,16.0
2,6.0,7.0,8.0,NaN,17.0
3,9.0,10.0,11.0,NaN,18.0


`dropna` es perfecto para eliminar rapidamente las filas con registros faltantes:

In [56]:
df.dropna()

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0


In [50]:
df[['a']].dropna()

,a
0,0.0
1,3.0
2,6.0
3,9.0


### Lidiando con valores nulos

Ya que hemos visto cómo funcionan las variable nulas, veamos cómo lidiar con
ellas. Usando la función `fillna` podremos reemplazarlas por el valor que
querramos, en este caso 0.

In [58]:
df.fillna(0)

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,0,16.0
2,6.0,7.0,8.0,0,17.0
3,9.0,10.0,11.0,0,18.0
4,0.0,13.0,14.0,0,19.0
5,0.0,0.0,0.0,10,0.0


Si quisieramos remplazar con el valor anterior usamos `method="ffill"`:

In [26]:
df.fillna(method="ffill")

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,1,16.0
2,6.0,7.0,8.0,1,17.0
3,9.0,10.0,11.0,1,18.0
4,9.0,13.0,14.0,1,19.0
5,9.0,13.0,14.0,10,19.0


Si quisieramos remplazar con el valor siguiente usamos
method="bfill":

In [59]:
df.fillna(method="bfill")

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,10,16.0
2,6.0,7.0,8.0,10,17.0
3,9.0,10.0,11.0,10,18.0
4,<NA>,13.0,14.0,10,19.0
5,<NA>,<NA>,<NA>,10,<NA>


El mismo ejercicio anterior se puede aplicar con las filas usando axis=1:

In [28]:
df.fillna(method="bfill",axis=1)

,a,b,c,d,e
0,0.0,1.0,2.0,1.0,15.0
1,3.0,4.0,5.0,16.0,16.0
2,6.0,7.0,8.0,17.0,17.0
3,9.0,10.0,11.0,18.0,18.0
4,13.0,13.0,14.0,19.0,19.0
5,10.0,10.0,10.0,10.0,<NA>


Podemos usar también una serie para reemplazar los valores de una columna en especifico, es importante que haya emparejamiento entre los índices:


In [29]:
fill = pd.Series([100, 101, 102])
fill

0    100
1    101
2    102
dtype: int64

In [34]:
df

,a,b,c,d,e
0,0.0,1.0,2.0,1,15.0
1,3.0,4.0,5.0,NaN,16.0
2,6.0,7.0,8.0,NaN,17.0
3,9.0,10.0,11.0,NaN,18.0
4,<NA>,13.0,14.0,NaN,19.0
5,<NA>,<NA>,<NA>,10,<NA>


In [37]:
df['d'] = df['d'].fillna(fill) 
df['d'] 


0      1.0
1    101.0
2    102.0
3      NaN
4      NaN
5     10.0
Name: d, dtype: float64

In [38]:
df

,a,b,c,d,e
0,0.0,1.0,2.0,1.0,15.0
1,3.0,4.0,5.0,101.0,16.0
2,6.0,7.0,8.0,102.0,17.0
3,9.0,10.0,11.0,NaN,18.0
4,<NA>,13.0,14.0,NaN,19.0
5,<NA>,<NA>,<NA>,10.0,<NA>


Una de las formas más usadas para reemplazar datos es usar el promedio de las columnas, esto se hace con la función `mean`. O si se quiere un mejor estimador, usamos `median`.

In [60]:
df.fillna(df.mean())

,a,b,c,d,e
0,0.0,1.0,2.0,1.0,15.0
1,3.0,4.0,5.0,5.5,16.0
2,6.0,7.0,8.0,5.5,17.0
3,9.0,10.0,11.0,5.5,18.0
4,4.5,13.0,14.0,5.5,19.0
5,4.5,7.0,8.0,10.0,17.0


In [39]:
df.fillna(df.median())

,a,b,c,d,e
0,0.0,1.0,2.0,1.0,15.0
1,3.0,4.0,5.0,101.0,16.0
2,6.0,7.0,8.0,102.0,17.0
3,9.0,10.0,11.0,55.5,18.0
4,4.5,13.0,14.0,55.5,19.0
5,4.5,7.0,8.0,10.0,17.0


Por último, Pandas también puede interporlar los valores faltanes calculando el valor que puede haber existido en el medio.

In [40]:
df_d = pd.concat([df[['d']], df[['d']].interpolate()],axis=1)
df_d.columns = ['d_antes','d_interpolado']
df_d


,d_antes,d_interpolado
0,1.0,1.000000
1,101.0,101.000000
2,102.0,102.000000
3,NaN,71.333333
4,NaN,40.666667
5,10.0,10.000000


In [77]:
df2 = pd.DataFrame([(0.0, np.nan, -1.0, 1.0),

                   (pd.NA, 2.0, pd.NA, np.nan),

                   (2.0, 3.0, np.nan, 9.0),

                   (np.nan, 4.0, -4.0, 16.0)],

                  columns=list('abcd'))
df2

,a,b,c,d
0,0.0,NaN,-1.0,1.0
1,<NA>,2.0,<NA>,NaN
2,2.0,3.0,NaN,9.0
3,NaN,4.0,-4.0,16.0


In [81]:
df2=df2.fillna(np.nan)

In [84]:
df2.interpolate()

,a,b,c,d
0,0.0,NaN,-1.0,1.0
1,1.0,2.0,-2.0,5.0
2,2.0,3.0,-3.0,9.0
3,2.0,4.0,-4.0,16.0


In [85]:
df2.interpolate(method='linear', limit_direction='forward', axis=0)

,a,b,c,d
0,0.0,NaN,-1.0,1.0
1,1.0,2.0,-2.0,5.0
2,2.0,3.0,-3.0,9.0
3,2.0,4.0,-4.0,16.0
